In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [34]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [35]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [36]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [37]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [5]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [6]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [7]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [8]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [9]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
#sf.bulk.Course__c.insert(course_data_records)

# Prepare ETL for the Student data

In [11]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head(30)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F
5,38,25006014,Dartling,Precious,Mariette,None,F
6,39,25007528,Dartling,Cindy,Patty,None,F
7,40,25003778,Dartling,Carlie,Raelene,None,F
8,41,25003605,Dartling,Clementine,Kendall,None,F
9,42,25002056,Dartling,Amirah,Angela,None,F


In [13]:
#rename columns from student table

student_df = student_data_df.rename(columns={
    'StudentID':'Student_Record_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
    'Gender': 'Gender__c',
}, inplace=True)

student_df

#Dropping ID_Student
student_df = student_data_df[['Student_Record_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c', 'Gender__c']]
student_df.head()

,Student_Record_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [14]:
student_data_records = student_df.to_dict('records')
student_data_records

[{'Student_Record_ID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_Record_ID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mari

In [18]:
for rec in student_data_records:
    #taking a key out of dictionary putting into new dic format
    record = {
        'Student_ID__c':rec['Student_Record_ID__c'],
        'Last_Name__c':rec['Last_Name__c'],
        'First_Name__c':rec['First_Name__c'],
        'Middle_Name__c':rec['Middle_Name__c'], 
        'Birth_Date__c':rec['Birth_Date__c'],
        'Gender__c': rec['Gender__c'],
    }
    
    try:
        sf.Student_ID__c.create(record)
    except Exception as e:
        print(e)

# Prepare ETL for the Staff data

In [19]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [20]:
staff_data_df.rename(columns={
    'EmployeeID':'Employee_ID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c', 
    'BirthDate':'Birth_Date__c',
}, inplace=True)

staff_data_df = staff_data_df[['Employee_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c']]
staff_data_df.head()

,Employee_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [21]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'Employee_ID__c': '000184220',
  'Last_Name__c': 'Luongo',
  'First_Name__c': 'Darick',
  'Middle_Name__c': 'Nico',
  'Birth_Date__c': datetime.date(1995, 7, 5)},
 {'Employee_ID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'First_Name__c': 'Gemini',
  'Middle_Name__c': 'Blair',
  'Birth_Date__c': datetime.date(1992, 4, 22)},
 {'Employee_ID__c': '000160655',
  'Last_Name__c': 'Williams',
  'First_Name__c': 'Dartanion',
  'Middle_Name__c': 'De Angelo',
  'Birth_Date__c': datetime.date(1993, 5, 21)},
 {'Employee_ID__c': '000159108',
  'Last_Name__c': 'Rodney',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Nicole',
  'Birth_Date__c': datetime.date(1994, 6, 7)}]

In [22]:
for rec in staff_data_records:
   
    record = {
        'Employee_ID__c':rec['Employee_ID__c'],
        'Last_Name__c':rec['Last_Name__c'],
        'First_Name__c':rec['First_Name__c'],
        'Middle_Name__c':rec['Middle_Name__c'], 
        'Birth_Date__c':str(rec['Birth_Date__c'])
    } 
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [23]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x0000035poe', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a004x0000035poo', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x0000035pot', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x0000035poy', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x0000035pp3', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x0000035pp8', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x0000035ppD', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x0000035poj', 'Course_Code__c': 'BC-WEBDEV'}]

In [24]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x0000035poe,BC-DATAVIZ
1,a004x0000035poo,BC-UIUX
2,a004x0000035pot,CIS-349
3,a004x0000035poy,CIS-405
4,a004x0000035pp3,CIS-438
5,a004x0000035pp8,CIS-430
6,a004x0000035ppD,CIS-435
7,a004x0000035poj,BC-WEBDEV


In [25]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [26]:
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [27]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x0000035poe
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x0000035poe
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x0000035poj
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x0000035poo


In [28]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df


,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x0000035poe
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x0000035poe
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x0000035poj
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x0000035poo


In [29]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c': 'a004x0000035poe'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'ID_Course__c': 'a004x0000035poe'},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x0000035poj'},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x0000035poo'}]

## Insert `Class` Records into Salesforce

In [30]:
for rec in class_data_records:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

# Class Participant

In [38]:
sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a004x0000035poe,BC-DATAVIZ
1,a004x0000035poo,BC-UIUX
2,a004x0000035pot,CIS-349
3,a004x0000035poy,CIS-405
4,a004x0000035pp3,CIS-438


In [39]:
sf_class_records = []

sf_class_data = sf.query('SELECT ID_Course__c, Name, Section__c FROM Class__c')

for row in sf_class_data['records']:
    
    rec = {
        'ID_Course__c': row['ID_Course__c'],
        'ID_Class__c': row['Name'],
        'Section__c': row['Section__c']
    }
    
    sf_class_records.append(rec)
    
# temporary workaround to strip last three characters from ID_Course__c
for rec in sf_class_records:
    rec['ID_Course__c'] = rec['ID_Course__c'][:-3]
    
sf_class_df = pd.DataFrame(sf_class_records)
sf_class_df.head()

,ID_Course__c,ID_Class__c,Section__c
0,a004x0000035poo,a014x0000095zRa,GWARL201905UIUX3
1,a004x0000035poj,a014x0000095zRV,GWARL201905WEB3
2,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0
3,a004x0000035poe,a014x0000095zRQ,GWDC201805DATA3


In [40]:
sf_class_xwalk = pd.merge(sf_class_df, sf_course_df)
sf_class_xwalk

,ID_Course__c,ID_Class__c,Section__c,Course_Code__c
0,a004x0000035poo,a014x0000095zRa,GWARL201905UIUX3,BC-UIUX
1,a004x0000035poj,a014x0000095zRV,GWARL201905WEB3,BC-WEBDEV
2,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,a004x0000035poe,a014x0000095zRQ,GWDC201805DATA3,BC-DATAVIZ


In [49]:
sf_student_records = []
sf_student_data = sf.query('SELECT Name, Student_ID__c FROM Student_ID__c')
for row in sf_student_data['records']:
    rec = {
        'ID_Student__c': row['Name'],
        'Student_ID__c': row['Student_ID__c']
    }
    sf_student_records.append(rec)
sf_student_xwalk = pd.DataFrame(sf_student_records)
sf_student_xwalk.head()

,ID_Student__c,Student_ID__c
0,a044x000002frbu,25002876
1,a044x000002fra3,25007185
2,a044x000002frbp,25006027
3,a044x000002frac,25004097
4,a044x000002frbf,25005443


In [50]:
query = '''

    SELECT
        s.StudentID AS Student_ID__c,
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cp.StartDate AS Start_Date__c,
        cp.EndDate AS End_Date__c
    FROM 
        classparticipant cp
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN student s
        ON cp.ID_Student = s.ID_Student

'''

classparticiant_df = pd.read_sql(query, conn)
classparticiant_df.head()

,Student_ID__c,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [51]:
#sf_class_xwalk
#sf_student_xwalk

class_participant_load = pd.merge(classparticiant_df, sf_class_xwalk, on=['Course_Code__c', 'Section__c'])

class_participant_load = pd.merge(class_participant_load, sf_student_xwalk)

class_participant_load = class_participant_load[['ID_Student__c','ID_Class__c','Start_Date__c','End_Date__c']]

class_participant_load['Start_Date__c'] = class_participant_load['Start_Date__c'].astype(str)

class_participant_load

class_participant_records = class_participant_load.to_dict(orient='records')
class_participant_records


[{'ID_Student__c': 'a044x000002frZj',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a044x000002frZo',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a044x000002frZt',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a044x000002frc4',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a044x000002frZy',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a044x000002fra3',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a044x000002fra8',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a044x000002fraD',
  'ID_Class__c': 'a014x0000095zRL',
  'Start_

In [52]:
for row in class_participant_records:
    try:
        sf.Class_Participant__c.create(row)
    except Exception as e:
        print(e)

# Staff Assignment

In [19]:
sf_staff_records = []
# NameError: name 'sf_Staff_data' is not defined
sf_staff_data = sf.query('SELECT Name, Last_Name__c, Middle_Name__c, First_Name__c, Birth_Date__c, Employee_ID__c FROM Staff__c')
for row in sf_staff_data['records']:
    rec = {
        'ID_Staff__c':row['Name'],
        'Last_Name__c': row['Last_Name__c'],
        'Middle_Name__c': row['Middle_Name__c'],
        'First_Name__c':row['First_Name__c'],
        'Birth_Date__c': row['Birth_Date__c'],
        'Employee_ID__c': row['Employee_ID__c']
    }
    sf_staff_records.append(rec)
sf_staff_records
sf_staff_xwalk = pd.DataFrame(sf_staff_records)
sf_staff_xwalk.head()

,ID_Staff__c,Last_Name__c,Middle_Name__c,First_Name__c,Birth_Date__c,Employee_ID__c
0,a054x000000auQT,Rodney,Nicole,Heather,1994-06-07,000159108
1,a054x000000auQO,Williams,De Angelo,Dartanion,1993-05-21,000160655
2,a054x000000auQE,Luongo,Nico,Darick,1995-07-05,000184220
3,a054x000000auQJ,Sanford,Blair,Gemini,1992-04-22,000130109


In [20]:
query  ='''
select
    co.CourseCode,
    cl.Section,
    s.EmployeeID,
    sa.role, 
    sa.startdate,
    sa.enddate
from
    staffassignment as sa
    inner join staff s
    on sa.ID_Staff = s.ID_staff
    inner join class cl
    on sa.ID_Class=cl.ID_Class
    inner join course co
    on co.ID_Course=cl.ID_Course
'''
staffassignment_df = pd.read_sql(query, conn)
staffassignment_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None


In [ ]:
#staffassignment_df
#sf_staff_xwalk
#sf_class_xwalk

In [21]:
print(staffassignment_df.columns)
print(sf_staff_xwalk.columns)
print(sf_class_xwalk.columns)

Index(['CourseCode', 'Section', 'EmployeeID', 'role', 'startdate', 'enddate'], dtype='object')
Index(['ID_Staff__c', 'Last_Name__c', 'Middle_Name__c', 'First_Name__c',
       'Birth_Date__c', 'Employee_ID__c'],
      dtype='object')
Index(['ID_Course__c', 'ID_Class__c', 'Section__c', 'Course_Code__c'], dtype='object')


In [22]:
staff_assignment_load_df = pd.merge(staffassignment_df, sf_class_xwalk, left_on=['CourseCode','Section'], right_on=['Course_Code__c','Section__c'])
staff_assignment_load_df.head()

,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Course__c,ID_Class__c,Section__c,Course_Code__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [24]:
staff_assignment_load_df = pd.merge(staff_assignment_load_df, sf_staff_xwalk, left_on='EmployeeID', right_on='Employee_ID__c')
staff_assignment_load_df.head()


,CourseCode,Section,EmployeeID,role,startdate,enddate,ID_Course__c,ID_Class__c,Section__c,Course_Code__c,ID_Staff__c,Last_Name__c,Middle_Name__c,First_Name__c,Birth_Date__c,Employee_ID__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a054x000000auQE,Luongo,Nico,Darick,1995-07-05,000184220
1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a054x000000auQT,Rodney,Nicole,Heather,1994-06-07,000159108
2,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,a004x0000035poe,a014x0000095zRL,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a054x000000auQO,Williams,De Angelo,Dartanion,1993-05-21,000160655


In [25]:
staff_assignment_load_df = staff_assignment_load_df[['ID_Staff__c','ID_Class__c','role','startdate','enddate']]
staff_assignment_load_df

,ID_Staff__c,ID_Class__c,role,startdate,enddate
0,a054x000000auQE,a014x0000095zRL,Teacher Assistant,2020-09-16,None
1,a054x000000auQT,a014x0000095zRL,Teacher Assistant,2020-09-16,None
2,a054x000000auQO,a014x0000095zRL,Instructor,2020-09-16,None


In [26]:
staff_assignment_load_df.rename(columns={
    'role':'Role__c',
    'startdate':'Start_Date__c',
    'enddate':'End_Date__c'
}, inplace=True)
staff_assignment_load_df['Start_Date__c'] = staff_assignment_load_df['Start_Date__c'].astype(str)
staff_assignment_load_df

,ID_Staff__c,ID_Class__c,Role__c,Start_Date__c,End_Date__c
0,a054x000000auQE,a014x0000095zRL,Teacher Assistant,2020-09-16,None
1,a054x000000auQT,a014x0000095zRL,Teacher Assistant,2020-09-16,None
2,a054x000000auQO,a014x0000095zRL,Instructor,2020-09-16,None


In [27]:
staff_assignment_records = staff_assignment_load_df.to_dict(orient='records')
staff_assignment_records

[{'ID_Staff__c': 'a054x000000auQE',
  'ID_Class__c': 'a014x0000095zRL',
  'Role__c': 'Teacher Assistant',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a054x000000auQT',
  'ID_Class__c': 'a014x0000095zRL',
  'Role__c': 'Teacher Assistant',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a054x000000auQO',
  'ID_Class__c': 'a014x0000095zRL',
  'Role__c': 'Instructor',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None}]

In [30]:
for rec in staff_assignment_records:
    try:
        sf.Staff_Assignment__c.create(rec)
    except Exception as e:
        print(e)

In [ ]:
#class_records = sf.query("SELECT Id FROM Course__c")
#recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
#recs_to_delete

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [ ]:
#for rec in recs_to_delete:
    #try:
        #sf.Course__c.delete(rec['Id'])
    #except Exception as e:
       # print(e)